In [ ]:
# !pip install bayesian-optimization

In [1]:
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# from bayes_opt import BayesianOptimization
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
# # Load the dataset
# reg_data = pd.read_csv('powerplant_energy_data.csv')
# print(reg_data.shape)

# # Assume the target variable is the last column
# target_column = reg_data.columns[-1]

# # Split the dataset into features (X) and target variable (y)
# X_reg = reg_data.drop(target_column, axis=1)
# y_reg = reg_data[target_column]

# # Split the data into training and testing sets
# X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)


(9568, 5)


In [5]:
# Load the dataset
reg_data = pd.read_csv('audiA1_price_data.csv')
print(reg_data.shape)

# Filter numerical columns
numerical_columns = reg_data.select_dtypes(include=['number']).columns

# Assume the target variable is the last column
target_column = numerical_columns[-1]

# Split the dataset into features (X) and target variable (y) using only numerical columns
X_reg = reg_data[numerical_columns[:-1]]  # Exclude the target column
y_reg = reg_data[target_column]

# Split the data into training and testing sets
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

(471, 16)


In [6]:
#@title HPO-GS-REG

dtrgs_s = time.time()
# Hyperparameter tuning grid for Decision Tree Regressor
dt_regressor_params = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Decision Tree Regressor on reg_data
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
dt_regressor = DecisionTreeRegressor(random_state=0)
grid_regressor = GridSearchCV(dt_regressor, dt_regressor_params, cv=3, scoring='neg_mean_squared_error')
grid_regressor.fit(X_reg_train, y_reg_train)
y_reg_pred = grid_regressor.best_estimator_.predict(X_reg_test)
r2 = r2_score(y_reg_test, y_reg_pred)
print("R2 score for Regression:", r2)
print("Best Parameters for Regression:", grid_regressor.best_params_)
dtrgs_e = time.time()
print(f"GS : DTR - {dtrgs_e-dtrgs_s}")

rfrgs_s = time.time()
# Hyperparameter tuning grid for Random Forest Regressor
rf_regressor_params = {
    'n_estimators': [10, 20, 30],
    'max_depth': [15, 20, 30, 50],
    'min_samples_leaf': [1, 2, 4, 8],
    'bootstrap': [True, False]
}
# Random Forest Regressor on reg_data
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
rf_regressor = RandomForestRegressor(random_state=0)
grid_regressor = GridSearchCV(rf_regressor, rf_regressor_params, cv=3, scoring='neg_mean_squared_error')
grid_regressor.fit(X_reg_train, y_reg_train)
y_reg_pred = grid_regressor.best_estimator_.predict(X_reg_test)
r2 = r2_score(y_reg_test, y_reg_pred)
print("R2 score for Regression:", r2)
print("Best Parameters for Regression:", grid_regressor.best_params_)
rfrgs_e = time.time()
print(f"GS : RFR - {rfrgs_e-rfrgs_s}")

gbrgs_s = time.time()
# Hyperparameter tuning grid for GBM Regressor
gbm_regressor_params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.05, 0.1, 0.2],
    'min_samples_split': [2, 3, 4]
}
# GBM Regressor on reg_data
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
gbm_regressor = GradientBoostingRegressor(random_state=0)
grid_regressor = GridSearchCV(gbm_regressor, gbm_regressor_params, cv=3, scoring='neg_mean_squared_error')
grid_regressor.fit(X_reg_train, y_reg_train)
y_reg_pred = grid_regressor.best_estimator_.predict(X_reg_test)
r2 = r2_score(y_reg_test, y_reg_pred)
print("R2 score for Regression:", r2)
print("Best Parameters for Regression:", grid_regressor.best_params_)
gbrgs_e = time.time()
print(f"GS : GBR - {gbrgs_e-gbrgs_s}")

R2 score for Regression: 0.9987358193835655
Best Parameters for Regression: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
GS : DTR - 0.5422148704528809
R2 score for Regression: 0.9987435820596765
Best Parameters for Regression: {'bootstrap': False, 'max_depth': 15, 'min_samples_leaf': 1, 'n_estimators': 10}
GS : RFR - 12.833272218704224
R2 score for Regression: 0.9990919217524548
Best Parameters for Regression: {'learning_rate': 0.05, 'min_samples_split': 2, 'n_estimators': 150}
GS : GBR - 9.10937762260437


In [ ]:
#@title HPO-BO-REG

# dtrbo_s = time.time()
# # Decision Tree Regressor
# def optimize_dtr(max_depth, min_samples_split, min_samples_leaf):
#     dtr = DecisionTreeRegressor(max_depth=int(max_depth), min_samples_split=int(min_samples_split),
#                                  min_samples_leaf=int(min_samples_leaf))
#     dtr.fit(X_train_reg, y_train_reg)
#     y_pred = dtr.predict(X_test_reg)
#     return r2_score(y_test_reg, y_pred)

# dtr_bounds = {'max_depth': (1, 30), 'min_samples_split': (2, 20),
#               'min_samples_leaf': (1, 20)}

# bayes_dtr = BayesianOptimization(f=optimize_dtr, pbounds=dtr_bounds, random_state=42)
# bayes_dtr.maximize(init_points=10, n_iter=10)
# print("Decision Tree Regressor: Best parameters -", bayes_dtr.max)
# print("R-squared Score:", bayes_dtr.max['target'])
# dtrbo_e = time.time()
# print(f"BO : DTR - {dtrbo_e-dtrbo_s}")
# print()

# rfrbo_s = time.time()
# # Random Forest Regressor
# def optimize_rfr(n_estimators, max_depth, min_samples_split, min_samples_leaf):
#     rfr = RandomForestRegressor(n_estimators=int(n_estimators), max_depth=int(max_depth),
#                                  min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
#     rfr.fit(X_train_reg, y_train_reg)
#     y_pred = rfr.predict(X_test_reg)
#     return r2_score(y_test_reg, y_pred)

# rfr_bounds = {'n_estimators': (5, 50), 'max_depth': (1, 30), 'min_samples_split': (2, 20),
#               'min_samples_leaf': (1, 20)}

# bayes_rfr = BayesianOptimization(f=optimize_rfr, pbounds=rfr_bounds, random_state=42)
# bayes_rfr.maximize(init_points=10, n_iter=10)
# print("Random Forest Regressor: Best parameters -", bayes_rfr.max)
# print("R-squared Score:", bayes_rfr.max['target'])
# rfrbo_e = time.time()
# print(f"BO : RFR - {rfrbo_e-rfrbo_s}")
# print()

# gbrbo_s = time.time()
# # Gradient Boosting Regressor
# def optimize_gbr(n_estimators, learning_rate, max_depth, min_samples_split, min_samples_leaf, subsample):
#     gbr = GradientBoostingRegressor(n_estimators=int(n_estimators), learning_rate=learning_rate,
#                                      max_depth=int(max_depth), min_samples_split=int(min_samples_split),
#                                      min_samples_leaf=int(min_samples_leaf), subsample=subsample)
#     gbr.fit(X_train_reg, y_train_reg)
#     y_pred = gbr.predict(X_test_reg)
#     return r2_score(y_test_reg, y_pred)

# gbr_bounds = {'n_estimators': (5, 50), 'learning_rate': (0.001, 1.0), 'max_depth': (1, 30),
#               'min_samples_split': (2, 20), 'min_samples_leaf': (1, 20), 'subsample': (0.1, 1.0)}

# bayes_gbr = BayesianOptimization(f=optimize_gbr, pbounds=gbr_bounds, random_state=42)
# bayes_gbr.maximize(init_points=10, n_iter=10)
# print("GBM Regressor: Best parameters -", bayes_gbr.max)
# print("R-squared Score:", bayes_gbr.max['target'])
# gbrbo_e = time.time()
# print(f"BO : GBR - {gbrbo_e-gbrbo_s}")

|   iter    |  target   | max_depth | min_sa... | min_sa... |
-------------------------------------------------------------
| 1         | 0.9874    | 11.86     | 19.06     | 15.18     |
| 2         | 0.9986    | 18.36     | 3.964     | 4.808     |
| 3         | 0.8945    | 2.684     | 17.46     | 12.82     |
| 4         | 0.9975    | 21.53     | 1.391     | 19.46     |
| 5         | 0.9992    | 25.14     | 5.034     | 5.273     |
| 6         | 0.999     | 6.319     | 6.781     | 11.45     |
| 7         | 0.999     | 13.53     | 6.533     | 13.01     |
| 8         | 0.9977    | 5.045     | 6.551     | 8.595     |
| 9         | 0.9875    | 14.23     | 15.92     | 5.594     |
| 10        | 0.9939    | 15.91     | 12.26     | 2.836     |
| 11        | 0.9875    | 22.92     | 15.89     | 16.73     |
| 12        | 0.9975    | 8.153     | 1.0       | 20.0      |
| 13        | 0.9988    | 8.53      | 1.0       | 2.0       |
| 14        | 0.9871    | 30.0      | 17.19     | 2.0       |
| 15    